In [1]:
import pandas as pd
import numpy as np
import csv

In [2]:
cali_layoffs = pd.read_csv('california_warn_raw.csv')
cali_population = pd.read_csv('county_population.csv', delimiter='\t', thousands=',')

- California WARN data: https://edd.ca.gov/Jobs_and_Training/Layoff_Services_WARN.htm
- population data from: https://www.census.gov/data/tables/time-series/demo/popest/2010s-counties-total.html#par_textimage_242301767

In [3]:
cali_layoffs.head()

,Notice Date,Effective Date,Received Date,Company,City,County,Employees,Layoff/Closure
0,10/26/2020,12/28/2020,11/10/2020,DAMAC Products LLC,La Mirada,Los Angeles County,79.0,Closure Permanent
1,10/27/2020,12/31/2020,11/10/2020,Gulfstream,Long Beach,Los Angeles County,608.0,Closure Permanent
2,10/26/2020,12/16/2020,11/10/2020,Renaissance Newport Beach Hotel,Newport Beach,Orange County,104.0,Closure Permanent
3,10/01/2020,10/05/2020,11/10/2020,Howmet Aerospace,Rancho Cucamonga,San Bernardino County,66.0,Layoff Temporary
4,10/26/2020,07/07/2020,11/10/2020,"Newport Diversified, Inc.",El Cajon,San Diego County,91.0,Layoff Temporary


In [4]:
cali_layoffs.tail()

,Notice Date,Effective Date,Received Date,Company,City,County,Employees,Layoff/Closure
9502,06/25/2015,08/24/2015,06/30/2015,"Intuit, Inc.",San Francisco,No County Name,86.0,Layoff Permanent
9503,06/25/2015,08/24/2015,06/30/2015,"Intuit, Inc.",Santa Monica,No County Name,49.0,Closure Permanent
9504,06/25/2015,08/24/2015,06/30/2015,"Intuit, Inc.",Venice,No County Name,11.0,Closure Permanent
9505,06/29/2015,08/28/2015,06/30/2015,"Safeway, Inc.",Pleasanton,No County Name,18.0,Layoff Unknown at this time
9506,06/30/2015,07/22/2015,06/30/2015,Medtronic Ablation Frontiers LLC,Carlsbad,No County Name,50.0,Closure Permanent


## California Layoffs

For the purposes of this analysis, we are selecting 2020 records by isolating the year on the `Notice date` column.
By choosing `Notice date` we make sure that these layoffs were issued after the start of shelter in place for 2020, as some layoff notices can be issues much earlier the layoff date. Here, we are also doing a bit of data cleaning to standardize county names.

In [16]:
cali_layoffs['Year'] = cali_layoffs['Notice Date'].str[-4:]
cali_layoffs['Layoff/Closure'] = cali_layoffs['Layoff/Closure'].str.replace('at thistime', 'at this time')
cali_layoffs['County'] = cali_layoffs['County'].str.strip()
cali_layoffs.loc[~(cali_layoffs['County'].str.endswith('County')),'County'] = cali_layoffs['County'] + " County"
cali_layoffs['County'] = cali_layoffs['County'].str.replace('No County Name County', 'No Name')

## Cleaning Layoff Types

Type Unknown and unknown at this time will be combined.

In [25]:
cali_layoffs['Layoff/Closure'].unique()

array(['Closure Permanent', 'Layoff Temporary', 'Closure Temporary',
       'Layoff Permanent', 'Layoff Type Unknown', 'Closure Type Unknown',
       'Closure Unknown at this time', 'Layoff Unknown at this time'],
      dtype=object)

In [34]:
cali_layoffs['Layoff/Closure'] = cali_layoffs['Layoff/Closure'].str.strip()
cali_layoffs['Layoff/Closure'] = cali_layoffs['Layoff/Closure'].str.lower()
cali_layoffs['Layoff/Closure'] = cali_layoffs['Layoff/Closure'].str.replace('unknown at this time', 'type unknown')
cali_layoffs['Layoff/Closure'].unique()

array(['closure permanent', 'layoff temporary', 'closure temporary',
       'layoff permanent', 'layoff type unknown', 'closure type unknown'],
      dtype=object)

In [17]:
cali_2020 = cali_layoffs[cali_layoffs['Year'] == '2020']
cali_2020['Employees'] = cali_2020['Employees'].astype('Int64')
cali_2020.tail()

<ipython-input-17-3004cf2829a9>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cali_2020['Employees'] = cali_2020['Employees'].astype('Int64')


,Notice Date,Effective Date,Received Date,Company,City,County,Employees,Layoff/Closure,Year
5636,01/03/2020,03/06/2020,01/09/2020,"Medtronic, Inc.",Goleta,Santa Barbara County,33,Layoff Permanent,2020
5637,01/06/2020,03/07/2020,01/08/2020,"Edmunds.com, Inc.",Santa Monica,Los Angeles County,122,Layoff Permanent,2020
5638,01/06/2020,03/09/2020,01/06/2020,SunSelect Produce (California) Inc.,Tehachapi,Kern County,236,Closure Permanent,2020
5639,01/03/2020,03/04/2020,01/03/2020,"Aramark Services, Inc.",Anaheim,Orange County,9,Closure Permanent,2020
5640,01/03/2020,03/04/2020,01/03/2020,"Aramark Services, Inc.",Rancho SantaMargarita,Orange County,13,Closure Permanent,2020


In [18]:
cali_layoffs.tail()

,Notice Date,Effective Date,Received Date,Company,City,County,Employees,Layoff/Closure,Year
9502,06/25/2015,08/24/2015,06/30/2015,"Intuit, Inc.",San Francisco,No Name,86.0,Layoff Permanent,2015
9503,06/25/2015,08/24/2015,06/30/2015,"Intuit, Inc.",Santa Monica,No Name,49.0,Closure Permanent,2015
9504,06/25/2015,08/24/2015,06/30/2015,"Intuit, Inc.",Venice,No Name,11.0,Closure Permanent,2015
9505,06/29/2015,08/28/2015,06/30/2015,"Safeway, Inc.",Pleasanton,No Name,18.0,Layoff Unknown at this time,2015
9506,06/30/2015,07/22/2015,06/30/2015,Medtronic Ablation Frontiers LLC,Carlsbad,No Name,50.0,Closure Permanent,2015


In [19]:
cali_layoffs['County'].unique()

array(['Los Angeles County', 'Orange County', 'San Bernardino County',
       'San Diego County', 'Sacramento County', 'Alameda County',
       'Santa Clara County', 'San Francisco County', 'Stanislaus County',
       'Contra Costa County', 'Riverside County', 'Solano County',
       'Ventura County', 'Monterey County', 'Placer County',
       'San Mateo County', 'Santa Barbara County', 'Fresno County',
       'Kings County', 'Kern County', 'Santa Cruz County',
       'Tehama County', 'Mendocino County', 'San Joaquin County',
       'Butte County', 'Sonoma County', 'Napa County', 'Calaveras County',
       'San Luis Obispo County', 'Siskiyou County', 'Yolo County',
       'Yuba County', 'Sutter County', 'Tulare County',
       'El Dorado County', 'Marin County', 'Mariposa County',
       'Imperial County', 'Shasta County', 'Madera County',
       'Del Norte County', 'Mono County', 'Merced County',
       'Humboldt County', 'San Benito County', 'Nevada County',
       'Tuolumne County',

In [20]:
years = ['2020', '2019']

## Extracting Only Necessary Records

Now that the data has been somewhat standardized, we will be selecting the records that have a notice date from the year 2020 or 2019. This will allow for comparisons between a pandemic and non-pandemic year in terms of layoff notices. 

In [21]:
cali_2020_2019 = cali_layoffs[cali_layoffs["Year"].isin(years)]
cali_2020_2019

,Notice Date,Effective Date,Received Date,Company,City,County,Employees,Layoff/Closure,Year
0,10/26/2020,12/28/2020,11/10/2020,DAMAC Products LLC,La Mirada,Los Angeles County,79.0,Closure Permanent,2020
1,10/27/2020,12/31/2020,11/10/2020,Gulfstream,Long Beach,Los Angeles County,608.0,Closure Permanent,2020
2,10/26/2020,12/16/2020,11/10/2020,Renaissance Newport Beach Hotel,Newport Beach,Orange County,104.0,Closure Permanent,2020
3,10/01/2020,10/05/2020,11/10/2020,Howmet Aerospace,Rancho Cucamonga,San Bernardino County,66.0,Layoff Temporary,2020
4,10/26/2020,07/07/2020,11/10/2020,"Newport Diversified, Inc.",El Cajon,San Diego County,91.0,Layoff Temporary,2020
...,...,...,...,...,...,...,...,...,...
6740,06/28/2019,08/28/2019,06/28/2019,"California Comfort Systems USA, Inc.",North Highlands,Sacramento County,15.0,Closure Permanent,2019
6741,06/28/2019,08/28/2019,06/28/2019,"California Comfort Systems USA, Inc.",Ontario,San Bernardino County,16.0,Closure Permanent,2019
6742,06/28/2019,08/28/2019,06/28/2019,"California Comfort Systems USA, Inc.",San Diego,San Diego County,84.0,Closure Permanent,2019
6743,06/28/2019,08/30/2019,06/28/2019,"Coloredge, Inc.",Burbank,Los Angeles County,32.0,Closure Permanent,2019


## More Data Cleaning

A few records have the incorrect county, based on the information in the city column. These cities include: Sacramento, Hayward, Indian Wells, Cerritos, Vista, Roseville, City of Industry, Del Mar, Walnut Creek, and Los Gatos.

In [22]:
dict_cities = {
    'Sacramento':'Sacramento County',
    'San Francisco':'San Francisco County',
    'Hayward':'Alameda County',
    'Indian Wells':'Riverside County',
    'Cerritos':'Los Angeles County',
    'Vista':'San Diego County',
    'Roseville':'Placer County',
    'City of Industry':'Los Angeles County',
    'Del Mar':'San Diego County',
    'Walnut Creek':'Contra Costa County',
    'Los Gatos':'Santa Clara County'
}

In [23]:
wrong_county = cali_2020_2019[(cali_2020_2019['City'] == 'Sacramento') & (cali_2020_2019['County'] == 'Del Norte County')]
cali_2020_2019.loc[1793,'County'] = 'Sacramento County'
wrong_county = cali_2020_2019[(cali_2020_2019['City'] == 'Hayward') & (cali_2020_2019['County'] == 'Contra Costa County')]
cali_2020_2019.loc[5749,'County'] = 'Alameda County'
wrong_county = cali_2020_2019[(cali_2020_2019['City'] == 'Indian Wells') & (cali_2020_2019['County'] == 'San Benito County')]
cali_2020_2019.loc[5246,'County'] = 'Riverside County'
wrong_county = cali_2020_2019[(cali_2020_2019['City'] == 'Cerritos') & (cali_2020_2019['County'] == 'Orange County')]
cali_2020_2019.loc[3168,'County'] = 'Los Angeles County'
wrong_county = cali_2020_2019[(cali_2020_2019['City'] == 'Vista') & (cali_2020_2019['County'] == 'San Francisco County')]
cali_2020_2019.loc[5202,'County'] = 'San Diego County'
wrong_county = cali_2020_2019[(cali_2020_2019['City'] == 'Roseville') & (cali_2020_2019['County'] == 'Sacramento County')]
cali_2020_2019.loc[4748,'County'] = 'Placer County'
wrong_county = cali_2020_2019[(cali_2020_2019['City'] == 'City of Industry') & (cali_2020_2019['County'] == 'Orange County')]
cali_2020_2019.loc[1772,'County'] = 'Los Angeles County'
wrong_county = cali_2020_2019[(cali_2020_2019['City'] == 'Del Mar') & (cali_2020_2019['County'] == 'Orange County')]
cali_2020_2019.loc[5190,'County'] = 'San Diego County'
wrong_county = cali_2020_2019[(cali_2020_2019['City'] == 'Walnut Creek') & (cali_2020_2019['County'] == 'Alameda County')]
cali_2020_2019.loc[4433,'County'] = 'Contra Costa County'
wrong_county = cali_2020_2019[(cali_2020_2019['City'] == 'Los Gatos') & (cali_2020_2019['County'] == 'Santa Cruz County')]
cali_2020_2019.loc[2648,'County'] = 'Santa Clara County'

/Users/dilcia_mercedes/.local/share/virtualenvs/cali_WARN-MY_IDcvv/lib/python3.8/site-packages/pandas/core/indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


## Next Steps
- [x] clean layoffs types
- [ ] calculate per capita instead of percentage
- [x] make sure this is the most up to date data or get the most up to date data

## Adding Population Data

The data has been grabbed from the census link above. These figures will be used to calculate per capita layoff numbers per county.

In [12]:
cali_population.rename(columns={'California':'Counties','39,512,223':'Population'}, inplace=True)
cali_population['County Names'] = cali_population['Counties'].str[1:]
cali_population['County Names'] = cali_population['County Names'].str.split(',').str[0]
cali_population.head(4)

,Counties,Population,County Names
0,".Alameda County, California",1671329,Alameda County
1,".Alpine County, California",1129,Alpine County
2,".Amador County, California",39752,Amador County
3,".Butte County, California",219186,Butte County


In [13]:
cali_merge = pd.merge(cali_2020_2019, cali_population, how="left", left_on="County", right_on="County Names")
cali_merge.drop(columns='Counties', inplace=True)
cali_merge.head()

,Notice Date,Effective Date,Received Date,Company,City,County,Employees,Layoff/Closure,Year,Population,County Names
0,10/26/2020,12/28/2020,11/10/2020,DAMAC Products LLC,La Mirada,Los Angeles County,79.0,Closure Permanent,2020,10039107,Los Angeles County
1,10/27/2020,12/31/2020,11/10/2020,Gulfstream,Long Beach,Los Angeles County,608.0,Closure Permanent,2020,10039107,Los Angeles County
2,10/26/2020,12/16/2020,11/10/2020,Renaissance Newport Beach Hotel,Newport Beach,Orange County,104.0,Closure Permanent,2020,3175692,Orange County
3,10/01/2020,10/05/2020,11/10/2020,Howmet Aerospace,Rancho Cucamonga,San Bernardino County,66.0,Layoff Temporary,2020,2180085,San Bernardino County
4,10/26/2020,07/07/2020,11/10/2020,"Newport Diversified, Inc.",El Cajon,San Diego County,91.0,Layoff Temporary,2020,3338330,San Diego County


In [46]:
layoffs_by_county = cali_merge.groupby(['County Names','Year']).agg('sum')
layoffs = layoffs_by_county.merge(cali_population, how="left", on="County Names")
layoffs.drop(columns=['Counties', 'Population_x'], inplace=True)
layoffs.rename(columns={"Population_y":"Population"}, inplace=True)
layoffs['Per Capita Laid Off'] = (layoffs['Employees']/layoffs['Population']) * 100000
layoffs.sort_values(by='Per Capita Laid Off', ascending=False)

,County Names,Employees,Population,Per Capita Laid Off
26,Mariposa County,2262.0,17203,13148.869383
51,San Francisco County,55796.0,881549,6329.313515
30,Mono County,879.0,14444,6085.571864
18,Inyo County,539.0,18039,2987.970508
49,San Diego County,75469.0,3338330,2260.681239
...,...,...,...,...
16,Imperial County,44.0,181215,24.280551
9,Del Norte County,6.0,27812,21.573422
74,Tehama County,10.0,65084,15.364759
78,Tuolumne County,1.0,54478,1.835603


,Notice Date,Effective Date,Received Date,Company,City,County,Employees,Layoff/Closure,Year,Population,County Names
6364,06/28/2019,08/28/2019,06/28/2019,"California Comfort Systems USA, Inc.",North Highlands,Sacramento County,15.0,Closure Permanent,2019,1552058,Sacramento County
6365,06/28/2019,08/28/2019,06/28/2019,"California Comfort Systems USA, Inc.",Ontario,San Bernardino County,16.0,Closure Permanent,2019,2180085,San Bernardino County
6366,06/28/2019,08/28/2019,06/28/2019,"California Comfort Systems USA, Inc.",San Diego,San Diego County,84.0,Closure Permanent,2019,3338330,San Diego County
6367,06/28/2019,08/30/2019,06/28/2019,"Coloredge, Inc.",Burbank,Los Angeles County,32.0,Closure Permanent,2019,10039107,Los Angeles County
6368,06/28/2019,08/31/2019,06/28/2019,Verity Business Services,San Jose,Santa Clara County,5.0,Layoff Permanent,2019,1927852,Santa Clara County
